# Transforms and Multi-Table Relational Databases
* This notebook shows how to run transforms directly on a mutli-table relational database 
* This notebook is discussed in ths [blog](https://gretel.ai/blog/transforms-and-multi-table-relational-databases)

## Capabilities
* This notebook can be run on any database SQLAlchemy supports such as Postgresql, SQLite or MySQL
* This notebook also contains instructions on how to transform data when the relational tables exist in CSV files.
* Referential integriety of primary and foreign keys will remain intact

## Limitations
* The primary and foreign keys in your database must be IDs
* Keys cannot be composite keys

## How to use this notebook on your own dataset
* Change the database connection string to refer to your database
* Alternatively, change the name and location of the CSV files where your data resides
* When viewing your data, change the table names used to your own table names
* Modify the location where you'd like your final synthetic data to be stored

## Our ecommerce database
* Execute the below cell to see a diagram of the database we'll be using in this blueprint. The lines in the diagram show connections between primary and foreign keys

In [ ]:
from IPython.display import Image
Image("https://gretel-blueprints-pub.s3.us-west-2.amazonaws.com/rdb/ecommerce_db.png",width = 600, height = 600)

## Getting started

In [ ]:
import os

!git clone https://github.com/gretelai/multi-table.git

os.chdir('./multi-table')
!pip install .

In [ ]:
# Specify your Gretel API key

from getpass import getpass
import pandas as pd
from gretel_client import configure_session, ClientConfig

pd.set_option('max_colwidth', None)

configure_session(ClientConfig(api_key=getpass(prompt="Enter Gretel API key"), 
                               endpoint="https://api.gretel.cloud"))

## Gather data and schema relationships directly from a database
* For demonstration purposes, we'll first grab our ecommerce SQLite database from S3
* This notebook can be run on any database SQLAlchemy supports such as Postgresql or MySQL
* For example, if you have a postgres database, simply swap the `sqlite:///` connection string for a `postgres://` one in the `create_engine` command
* Using SQLAlchemy's reflection extension, we will crawl the schema, gather table data and produce a list of relationships by table primary key.

In [ ]:
from sqlalchemy import create_engine
import multi_table.rdb_util as rdb

!wget https://gretel-blueprints-pub.s3.amazonaws.com/rdb/ecom.db
    
engine = create_engine("sqlite:///ecom.db")

rdb_config = rdb.crawl_db(engine)

## Alternatively, specify primary/foreign key relationships and locations of data csv files 
* This is an alternative to the above three cells that work directly with a database
* First, assign `base_path` to the directory where the csv files are located.
* Then, add a name/key pair for each table name/filename to `rdb_config["table_files"]`
* Add all primary keys for each table to `rdb_config["primary_keys"]`
* Add all foreign key/primary keys that connect to the same set under `rdb_config["relationshipts"]`

In [ ]:
# base_path is the directory where your csv files can be found
base_path = "https://gretel-blueprints-pub.s3.amazonaws.com/rdb/"

rdb_config = {
   "table_files": {
      "users": base_path + "users.csv",

      "order_items": base_path + "order_items.csv",
       
      "events": base_path + "events.csv",
       
      "inventory_items": base_path + "inventory_items.csv",  
       
      "products": base_path + "products.csv",
       
      "distribution_center": base_path + "distribution_center.csv"
   },
    
  # List the primary keys for each table
    
   "primary_keys": {
      "users": "id",

      "order_items": "id",
       
      "events": "id",
       
      "inventory_items": "id",  
       
      "products": "id",
       
      "distribution_center": "id"
   },

  # List the (table, field) relationships between primary and foreign keys  
   "relationships": [
          [("users","id"),
           ("order_items","user_id"),
           ("events","user_id")
          ],         
       
          [("inventory_items","id"),
           ("order_items","inventory_item_id")  
          ],         

          [("products","id"),
           ("inventory_items","product_id")
          ],                

          [("distribution_center","id"),
           ("products","distribution_center_id"),
           ("inventory_items", "product_distribution_center_id")
          ]             
   ]
}

# Gather the table data using the filenames entered above

rdb_config["table_data"] = {}
for table in rdb_config["table_files"]:
    filename = rdb_config["table_files"][table]
    df = pd.read_csv(filename)
    rdb_config["table_data"][table] = df

## Take a look at your data by joining two tables
* Note that every record in the table "order_items" matches to an entry in the table "users"
* An "inner" join will take the intersection of two tables

In [ ]:
pd.set_option("display.max_columns", None)

table1 = "order_items"
table2 = "users"
table1_key = "user_id"
table2_key = "id"
df1 = rdb_config["table_data"][table1]
df2 = rdb_config["table_data"][table2]

joined_data = df1.join(df2.set_index(table2_key), how='inner', on=table1_key, lsuffix='_order_items', rsuffix='_users')
print("Number of records in order_items table is " + str(len(df1)))
print("Number of records in user table is " + str(len(df2)))
print("Number of records in joined data is " + str(len(joined_data)))

joined_data.head()

## Define your transform policies
* Define one policy per table which transforms any PII or sensitive information that could be used to reidentify a user.
* You needn't include a transform for any of the primary/foreign key combinations. We'll be handling those seperately in order to maintain referential integrity.
* However, if a table contains a primary key that does not match to a foreign key, that field should be included in the transforms.
* Note the tables inventory_items, products and distribution center contain only public information so there will be no transformation.
* To run this notebook on a different database, simply enter the table names and policy files below. We will assume all policy files are located in the `policy_dir` defined below

In [ ]:
policy_dir = "https://gretel-blueprints-pub.s3.amazonaws.com/rdb/"

transform_policies = {}
transform_policies["users"] = policy_dir + "users_policy.yaml"
transform_policies["order_items"] = policy_dir + "order_items_policy.yaml"
transform_policies["events"] = policy_dir + "events_policy.yaml"
transform_policies["inventory_items"] =  None  
transform_policies["products"] = None
transform_policies["distribution_center"] = None

## Policy detail
* Let's take a detailed look at the transforms for the users table.
* Within the `rules` section, we define each type of transformation we want, each one beginning with `- name`.
* We start by replacing any field classified as a person name or email address with a fake version.
* Note, we choose to leave "city", "state", "country" and "zip" as is since it's public knowledge that this database is about user ecommerce transactions in Arizona.
* We then transform the "created_at" timestamp using a random date shift.
* And finally, we transform the numeric fields of age, latitude and longitude with a random numeric shift.
* Note, we do not transform "id" because it is a primary key that matches to a foreign key. We'll take care of that later.

In [ ]:
from smart_open import open

policy_file = transform_policies["users"]
yaml_file = open(policy_file, "r")
policy = yaml_file.read()
yaml_file.close()
print(policy)

## Create transformed data

In [ ]:
import multi_table.transform_models as tm
from gretel_client.projects import create_or_get_unique_project

# Designate a project
project = create_or_get_unique_project(name="rdb-transforms")

# Transform your tables
transformed_tables, errors = tm.transform_tables(rdb_config, project, transform_policies)

# Tranform your primary/foreign keys
if errors == False:
    transformed_tables = tm.transform_keys(transformed_tables, rdb_config)

## View the transformed content
* We'll again join the order_items and users tables

In [ ]:
pd.set_option("display.max_columns", None)

table1 = "order_items"
table2 = "users"
table1_key = "user_id"
table2_key = "id"
df1 = transformed_tables[table1]
df2 = transformed_tables[table2]

joined_data = df1.join(df2.set_index(table2_key), how='inner', on=table1_key, lsuffix='_order_items', rsuffix='_users')
print("Number of records in order_items table is " + str(len(df1)))
print("Number of records in user table is " + str(len(df2)))
print("Number of records in joined data is " + str(len(joined_data)))

joined_data.head()

## Save the transformed data back into a database
* Here, we're saving the data into an sqlite database called ecom_xf
* To save into a postgres database, use type="postgres"

In [ ]:
# Save the new data to ecom_xf using the schema in ecom
rdb.save_to_rdb("ecom", "ecom_xf", transformed_tables, engine, type="sqlite")

## Alterntively, save the transformed content into CSV files

In [ ]:
# Change final_dir to be the location where you'd like your csv files saved
final_dir = "./"
for table in transformed_tables:
    df = transformed_tables[table]
    filename = final_dir + table + '_transform.csv'
    df.to_csv(filename, index=False, header=True)